In [1]:
import os
import argparse
import json
import random
import tqdm.notebook as tqdm
import re
import razdel
import nltk

In [2]:
from _jsonnet import evaluate_file as jsonnet_evaluate_file
from transformers import BertTokenizer, EncoderDecoderModel, AutoModelForSequenceClassification, logging
import torch
import wandb

from readers.ria_reader import ria_reader
from readers.tg_reader import tg_reader
from custom_datasets.gen_title_dataset import GenTitleDataset
from utils.gen_title_calculate_metrics import print_metrics, first_sent, postprocess
from utils.training_utils import init_wandb

## Using clustering

In [3]:
logging.set_verbosity_info()

In [4]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

In [20]:
init_wandb('TRASH', config, run_id='1d4cpfah')

In [5]:
tokenizer_model_path = config["tokenizer_model_path"]
tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False, do_basic_tokenize=False)

Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/added_tokens.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/special_tokens_map.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer_config.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer.json. We won't load it.
loading file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [7]:
eval_model_file = '../../models/lenta_ria_cross_style_from_pretrained/checkpoint-3500/'

In [6]:
max_tokens_text = config["max_tokens_text"]
max_tokens_title = config["max_tokens_title"]
setattr(tokenizer, 'max_tokens_text', max_tokens_text)

batch_size = config["batch_size"]

# print("Loading model...")
# model = EncoderDecoderModel.from_pretrained(eval_model_file)
# model.eval()
# model.cuda();

In [7]:
import json

def reader(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line.strip())

In [8]:
records = [r for r in tqdm.tqdm(reader('../../datasets/full_lenta_ria.test.jsonl'))]
lenta_records = [
    {'text': r['lenta_text'], 'title': r['lenta_title'], 'agency': 'lenta.ru', 'date': r['lenta_date']} 
    for r in records
]

ria_records = [
    {'text': r['ria_text'], 'title': r['ria_title'], 'agency': 'РИА Новости', 'date': r['lenta_date']} 
    for r in records
]
records = lenta_records + ria_records

0it [00:00, ?it/s]

In [11]:
test_dataset = GenTitleDataset(
    records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

# Style vs No style

In [9]:
test_dataset = GenTitleDataset(
    records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

ria_dataset = GenTitleDataset(
    ria_records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

lenta_dataset = GenTitleDataset(
    lenta_records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

print('Dataset size:', len(test_dataset))

Dataset size: 4000


In [10]:
agency_list = ["РИА Новости", "lenta.ru"]
agency_to_special_token_id = {a: tokenizer.vocab[f'[unused{i+1}]'] for i, a in enumerate(agency_list)}
agency_to_discr_target = {a: i for i, a in enumerate(sorted(agency_list))}

In [11]:
vanilla_model = EncoderDecoderModel.from_pretrained('../../models/gen_title_lenta_ria/checkpoint-6000/')
vanilla_model.eval()
vanilla_model.cuda()

style_model = EncoderDecoderModel.from_pretrained('../../models/lenta_ria_cross_style_from_pretrained/checkpoint-3500/')
style_model.eval()
style_model.cuda()

discriminator = AutoModelForSequenceClassification.from_pretrained(
    '/home/aobuhtijarov/models/agency_discriminator_tuned/checkpoint-4000/'
)
discriminator.eval()
discriminator.cuda();

loading configuration file ../../models/gen_title_lenta_ria/checkpoint-6000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/home/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.2,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.2,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,

loading weights file ../../models/lenta_ria_cross_style_from_pretrained/checkpoint-3500/pytorch_model.bin
All model checkpoint weights were used when initializing EncoderDecoderModel.

All the weights of EncoderDecoderModel were initialized from the model checkpoint at ../../models/lenta_ria_cross_style_from_pretrained/checkpoint-3500/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use EncoderDecoderModel for predictions without further training.
loading configuration file /home/aobuhtijarov/models/agency_discriminator_tuned/checkpoint-4000/config.json
Model config BertConfig {
  "_name_or_path": "/home/aobuhtijarov/master-thesis/src/bert_ft_on_tg_text/checkpoint-2000/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
 

In [12]:
refs_ria, refs_lenta, hyps_ria, hyps_lenta = [], [], [], []

In [ ]:
with open('on_ria_2.txt', 'w') as f:
    for i in tqdm.trange(0, len(ria_dataset), batch_size):
        data = ria_dataset[i]
        
        for k in tuple(data.keys()):
            if k not in ('input_ids', 'attention_mask'):
                del data[k]
            else:
                data[k] = data[k].unsqueeze(0)

        for j in range(i + 1, min(i + batch_size, len(test_dataset))):
            for k in data.keys():
                data[k] = torch.cat((data[k], test_dataset[j][k].unsqueeze(0)), dim=0)

        data['input_ids'] = data['input_ids'].cuda()
        data['attention_mask'] = data['attention_mask'].cuda()

        vanilla_output_ids = vanilla_model.generate(
            **data,
            decoder_start_token_id=vanilla_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )

        vanilla_preds = [
            tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in vanilla_output_ids
        ]
        
        style_output_ids = style_model.generate(
            **data,
            decoder_start_token_id=style_model.config.decoder.pad_token_id,
            min_length=7,
            max_length=20,
            num_beams=6
        )

        style_preds = [
            tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in style_output_ids
        ]
        
        for j in range(i, min(i + batch_size, len(test_dataset))):
            ria_ref = ria_dataset.get_strings(j)['title']
            lenta_ref = lenta_dataset.get_strings(j)['title']
            va_hyp = vanilla_preds[j-i]
            st_hyp = style_preds[j-i]
            
            inp_va = tokenizer(va_hyp, 
                add_special_tokens=True, max_length=max_tokens_title,
                padding='max_length', truncation=True
            )
            inp_st = tokenizer(st_hyp, 
                add_special_tokens=True, max_length=max_tokens_title,
                padding='max_length', truncation=True
            )
            logits_va = discriminator(input_ids=torch.LongTensor(inp_va['input_ids']).cuda().unsqueeze(0), 
                               attention_mask=torch.LongTensor(inp_va['attention_mask']).cuda().unsqueeze(0))[0]
            logits_st = discriminator(input_ids=torch.LongTensor(inp_st['input_ids']).cuda().unsqueeze(0), 
                               attention_mask=torch.LongTensor(inp_st['attention_mask']).cuda().unsqueeze(0))[0]            
            va_pred = torch.argmax(logits_va).item()
            st_pred = torch.argmax(logits_st).item()
            
            refs_ria.append(ria_ref)
            refs_lenta.append(lenta_ref)
            hyps_ria.append(va_hyp)
            hyps_lenta.append(st_hyp)
            
            if not (va_pred != st_pred and \
                    va_pred == agency_to_discr_target['РИА Новости'] and \
                    min(logits_va.max().item(), logits_st.max().item()) >= 3):
                continue
            
            ria_ref, va_hyp = postprocess(ria_ref, va_hyp, 'ru', is_multiple_ref=True, lower=True)
            lenta_ref, st_hyp = postprocess(lenta_ref, st_hyp, 'ru', is_multiple_ref=True, lower=True)
            
            f.write('RIA text: ' + ria_dataset.get_strings(j)['text'] + '\n\n')
            f.write('RIA Ref headline: ' + ria_ref[0] + '\n')
            f.write('Lenta Ref headline: ' + lenta_ref[0] + '\n\n')
            
            f.write('RIA Hyp headline: ' + va_hyp + '\n')
            f.write('Lenta Hyp headline: ' + st_hyp + '\n')
            
            f.write('\n\n' + '-' * 50 + '\n')
        

  0%|          | 0/500 [00:00<?, ?it/s]

#### Cross Rouge

In [ ]:
from utils.gen_title_calculate_metrics import calc_metrics

In [ ]:
set(type(x) for x in refs_ria), set(type(x) for x in refs_lenta), set(type(x) for x in hyps_ria), set(type(x) for x in hyps_lenta), 



In [34]:
# refs_ria = [list(x) for x in refs_ria]
# refs_lenta = [list(x) for x in refs_lenta]
# hyps_ria = [list(x) for x in hyps_ria]
# hyps_lenta = [list(x) for x in hyps_lenta]

In [93]:
refs_ria[0], refs_lenta[0], hyps_ria[0], hyps_lenta[0]

('медведев поздравил коморовского с победой на выборах президента польши',
 'медведев поздравил коморовского с победой на выборах',
 'медведев поздравил коморовского с успешным проведением выборов',
 'медведев поздравил коморовский с победой на выборах')

In [92]:
refs_ria = [x[0] for x in refs_ria]
refs_lenta = [x[0] for x in refs_lenta]
hyps_ria = [x[0] for x in hyps_ria]
# hyps_lenta = [x[0] for x in hyps_lenta]

In [ ]:
refs_ria = [[x] for x in refs_ria]
refs_ria_vs_refs_lenta = calc_metrics(refs_ria, refs_lenta, language='ru')
refs_ria_vs_hyps_ria = calc_metrics(refs_ria, hyps_ria, language='ru')
refs_ria_vs_hyps_lenta = calc_metrics(refs_ria, hyps_lenta, language='ru')

refs_lenta = [[x] for x in refs_lenta]
refs_lenta_vs_hyps_ria = calc_metrics(refs_lenta, hyps_ria, language='ru')
refs_lenta_vs_hyps_lenta = calc_metrics(refs_lenta, hyps_lenta, language='ru')

hyps_ria = [[x] for x in hyps_ria]
hyps_ria_vs_hyps_lenta = calc_metrics(hyps_ria, hyps_lenta, language='ru')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
row1 = {
    'id': 'RefsRIA',
    'RefsRIA': np.nan,
    'RefsLenta': scalar_from_metrics(refs_ria_vs_refs_lenta),
    'HypsRIA': scalar_from_metrics(refs_ria_vs_hyps_ria),
    'HypsLenta': scalar_from_metrics(refs_ria_vs_hyps_lenta),
}

row2 = {
    'id': 'RefsLenta', 
    'RefsRIA': np.nan,
    'RefsLenta': np.nan,
    'HypsRIA': scalar_from_metrics(refs_lenta_vs_hyps_ria),
    'HypsLenta': scalar_from_metrics(refs_lenta_vs_hyps_lenta),
}

row3 = {
    'id': 'HypsRIA',
    'RefsRIA': np.nan,
    'RefsLenta': np.nan,
    'HypsRIA': np.nan,
    'HypsLenta': scalar_from_metrics(hyps_ria_vs_hyps_lenta),
}

row4 = {
    'id': 'HypsLenta',
    'RefsRIA': np.nan,
    'RefsLenta': np.nan,
    'HypsRIA': np.nan,
    'HypsLenta': np.nan
}

In [ ]:
np.mean([len(x.split()) for x in refs_ria])

In [ ]:
np.mean([len(x.split()) for x in refs_lenta])

In [ ]:
np.mean([len(x.split()) for x in hyps_ria])

In [ ]:
np.mean([len(x.split()) for x in hyps_lenta])

In [ ]:
def scalar_from_metrics(m):
    return (m['rl'] + m['r2'] + m['rl']) / 3

In [20]:
df = pd.DataFrame(columns=['id', 'RefsRIA', 'RefsLenta', 'HypsRIA', 'HypsLenta'])
df = df.append(row1, ignore_index=True)
df = df.append(row2, ignore_index=True)
df = df.append(row3, ignore_index=True)
df = df.append(row4, ignore_index=True)
df.index = df['id']
df = df.drop('id', axis=1)
df.round(2)

,RefsRIA,RefsLenta,HypsRIA,HypsLenta
id,,,,
RefsRIA,NaN,0.32,0.34,0.35
RefsLenta,NaN,NaN,0.40,0.35
HypsRIA,NaN,NaN,NaN,0.49
HypsLenta,NaN,NaN,NaN,NaN


# VANILA

In [12]:
hyps = []
refs = []

table = wandb.Table(columns=['Reference', 'Prediction', 'Reference processed', 'Prediction processed'])

for i in tqdm.trange(0, len(test_dataset), batch_size):
    data = test_dataset[i]
    for k in tuple(data.keys()):
        if k not in ('input_ids', 'attention_mask'):
            del data[k]
        else:
            data[k] = data[k].unsqueeze(0)

    for j in range(i + 1, min(i + batch_size, len(test_dataset))):
        for k in data.keys():
            data[k] = torch.cat((data[k], test_dataset[j][k].unsqueeze(0)), dim=0)

    data['input_ids'] = data['input_ids'].cuda()
    data['attention_mask'] = data['attention_mask'].cuda()

    output_ids = model.generate(
        **data,
        decoder_start_token_id=model.config.decoder.pad_token_id,
        min_length=7,
        max_length=20,
        num_beams=6
    )

    preds = [
        tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in output_ids
    ]

    for j in range(i, min(i + batch_size, len(test_dataset))):
        if j < 2000:
            ref = ' s_s '.join([test_dataset.get_strings(j)['title'], 
                                test_dataset.get_strings(j + 2000)['title']])
        else:
            ref = ' s_s '.join([test_dataset.get_strings(j)['title'], 
                                test_dataset.get_strings(j - 2000)['title']])   
            
        ref = test_dataset.get_strings(j)['title']

        hyp = preds[j - i]
        
        if j % 500 == 0:
            ref_before = ref
            hyp_before = hyp        
        
        ref, hyp = postprocess(ref, hyp, 'ru', is_multiple_ref=True, lower=True)
        
        refs.append(ref)
        hyps.append(hyp)
        
        if j % 500 == 0:
            table.add_data(ref_before, hyp_before, ref, hyp)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
with open('style.txt', 'w') as f:
    for i in range(len(refs)):
        
        f.write('Ref: ' + ' s_s '.join(refs[i]) + '\n')
        f.write('Hyp: ' + hyps[i] + '\n')
        f.write('\n\n' + '-' * 40 + '\n')

In [23]:
!head -n 5 vanilla.txt

Ref: медведев поздравил коморовского с победой на выборах s_s медведев поздравил коморовского с победой на выборах президента польши
Hyp: медведев поздравил коморовского с победой на выборах


----------------------------------------


In [13]:
wandb.run.summary.update({'Examples': table})

In [14]:
print_metrics(refs, hyps, language='ru', are_clusters_used=False)

{'count': 4000, 'bleu': 0.4853310648776669, 'r1': 0.37826795698035687, 'r2': 0.18936949952669432, 'rl': 0.34854062601969343, 'duplicate_ngrams': {1: 0.01102777492379876, 2: 0.0003968411444898607, 3: 0.00014151610925043634, 4: 5.8139534883720933e-05}}


In [15]:
wandb.finish()

BLEU,48.53
V2 ROUGE-2-F,27.28
Count,4000
Dup 2-grams,0.04
V2 ROUGE-1-F,47.46
V2 ROUGE-L-F,44.33
V2 BLEU,64.96
ROUGE-2-F,18.94
ROUGE-L-F,34.85
Dup 1-grams,1.1
Dup 3-grams,0.01


# Direct

In [ ]:
logging.set_verbosity_info()

In [3]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

In [4]:
tokenizer_model_path = config["tokenizer_model_path"]
tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False, do_basic_tokenize=False)

In [5]:
eval_model_file = '../../models/gen_title_lenta_ria/checkpoint-5000/'

In [6]:
max_tokens_text = config["max_tokens_text"]
max_tokens_title = config["max_tokens_title"]
setattr(tokenizer, 'max_tokens_text', max_tokens_text)

batch_size = config["batch_size"]

print("Loading model...")
model = EncoderDecoderModel.from_pretrained(eval_model_file)
model.eval();

Loading model...


In [7]:
import json

def reader(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line.strip())

In [8]:
records = [r for r in tqdm.tqdm(reader('../../datasets/full_lenta_ria.test.jsonl'))]
lenta_records = [
    {'text': r['lenta_text'], 'title': r['lenta_title'], 'agency': 'lenta.ru', 'date': r['lenta_date']} 
    for r in records
]

ria_records = [
    {'text': r['ria_text'], 'title': r['ria_title'], 'agency': 'РИА Новости', 'date': r['lenta_date']} 
    for r in records
]
records = lenta_records + ria_records

0it [00:00, ?it/s]

In [9]:
test_dataset = GenTitleDataset(
    records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

print('Dataset size:', len(test_dataset))

Dataset size: 4000


In [10]:
model.cuda();

In [11]:
hyps = []
refs = []
table = wandb.Table(columns=['Reference', 'Prediction', 'Reference processed', 'Prediction processed'])

for i in tqdm.trange(0, len(test_dataset), batch_size):
    data = test_dataset[i]
    for k in tuple(data.keys()):
        if k not in ('input_ids', 'attention_mask'):
            del data[k]
        else:
            data[k] = data[k].unsqueeze(0)

    for j in range(i + 1, min(i + batch_size, len(test_dataset))):
        for k in data.keys():
            data[k] = torch.cat((data[k], test_dataset[j][k].unsqueeze(0)), dim=0)

    data['input_ids'] = data['input_ids'].cuda()
    data['attention_mask'] = data['attention_mask'].cuda()

    output_ids = model.generate(
        **data,
        decoder_start_token_id=model.config.decoder.pad_token_id,
        min_length=7,
        max_length=20,
        num_beams=6
    )

    preds = [
        tokenizer.decode(first_sent(x, tokenizer.sep_token_id), skip_special_tokens=True) for x in output_ids
    ]

    for j in range(i, min(i + batch_size, len(test_dataset))):
        ref = test_dataset.get_strings(j)['title']
        hyp = preds[j - i]
        
        if j % 500 == 0:
            ref_before = ref
            hyp_before = hyp        
        
        ref, hyp = postprocess(ref, hyp, 'ru', False, False, False, lower=True)
        
        refs.append(ref)
        hyps.append(hyp)
        
        if j % 500 == 0:
            table.add_data(ref_before, hyp_before, ref, hyp)
            

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
wandb.run.summary.update({'Examples': table})

In [ ]:
print_metrics(refs, hyps, language='ru', are_clusters_used=False)

In [ ]:
for i, (ref, hyp) in enumerate(zip(gold, pred)):

    if i % 500 == 0:
        ref_before = ref
        hyp_before = hyp

    if max_count is not None and i >= max_count:
        break

    ref, hyp = postprocess(ref, hyp, language, is_multiple_ref, detokenize_after, tokenize_after, lower)
    if not hyp:
        print("Empty hyp for ref: ", ref)
        continue
    if not ref:
        continue

    if i % 500 == 0:
        table.add_data(ref_before, hyp_before, ref, hyp)

    refs.append(ref)
    hyps.append(hyp)

if are_clusters_used:
    wandb.run.summary.update({'Examples with multiple references': table})
else:
    wandb.run.summary.update({'Examples': table})

print_metrics(refs, hyps, language=language, are_clusters_used=are_clusters_used)